# Getting Started

## Start the docker container on Jetson

1. Execute "code ." in a wsl shell and remote connect to jetson
2. Open a terminal on the jetson from with VS Code and "source enable.sh" from within the docker directory
    check that a container with image geoc1234/jetbot:jupyter-0.4.2-32.4.4.4 is running

## Start ROSCORE & Other Needed ROS Nodes, etc

1. ssh into jetson from MSI or open another terminal window in VS Code and execute: sudo docker exec -it jetbot_jupyter bash
   that will open a linux shell inside the container.
   Use ipconfig to find the IP address of Jetbot and export it as shown below
   -  export ROS_MASTER_URI=http://10.0.0.66:11311
   -  export ROS_IP=10.0.0.66
2. execute 'roscore'
3. Repeat (1) and 'roslaunch' bones_move ard.launch
4. Repeat (1) and 'roslaunch' bones_move bones.launch
5. Repeat (1) and 'roslaunch' robot_localization ekf_template.launch

### the following code is needed to initialize communication with "bones"

In [1]:
import sys
sys.path.insert(0, '/opt/ros/noetic/lib/python3/dist-packages')
import rospy
rospy.init_node('motion_control')

## create an instance of a robot
from jetbot import Bones
robot = Bones()
robot.observe(robot.fwd_cb,'fwdspd')
robot.observe(robot.ang_cb,'angspd')

## create an auto recurring timer and associated callback which triggers ever 250ms. The purpose is to throttle the transfer of twist 
## commands to the arduino.  
from jetbot import Mytimer

def ard_callback():
    robot.twist()

ard_timer = Mytimer(0.25, ard_callback)
ard_timer.start()

### Check that topics can be published and subscribed

In [2]:
import jupyros
from nav_msgs.msg import Odometry

odom_msg = Odometry()


In [3]:
def odom_cb(msg):
    odom_msg = msg
    
jupyros.subscribe('/odometry/filtered', Odometry, odom_cb)

### Create widgets for controlling the motors

In [4]:
import ipywidgets.widgets as widgets
from IPython.display import display

# create two sliders with range [-0.2, 0.2]
left_slider = widgets.FloatSlider(description='left', min=-0.2, max=0.2, step=0.02, orientation='vertical')
right_slider = widgets.FloatSlider(description='right', min=-0.4, max=0.4, step=0.02, orientation='vertical')

# create a horizontal box container to place the sliders next to each other
slider_container = widgets.HBox([left_slider, right_slider])

# display the container in this cell's output
display(slider_container)

import traitlets
left_link = traitlets.link((left_slider, 'value'), (robot, 'fwdspd'))
right_link = traitlets.link((right_slider, 'value'), (robot, 'angspd'))

### Visualization is accomplished through rviz running on the MSI Laptop